In [0]:
#https://medium.com/swlh/simple-transformers-multi-class-text-classification-with-bert-roberta-xlnet-xlm-and-8b585000ce3a

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/"My Drive"/"Colab Notebooks"/shared_task/xlnet

/content/drive/My Drive/Colab Notebooks/shared_task/xlnet


In [3]:
try:
 %tensorflow_version 2.x  #gpu
except Exception:
 pass
import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.x  #gpu`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
2.1.0


In [0]:
%%capture
!apt-get --purge -y remove 'cuda*'
!apt-get --purge -y remove 'nvidia*'
!apt autoremove -y
!apt-get clean

!reboot

In [0]:
%%capture
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1810/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-1=10.1.168-1

In [6]:
%%writefile setup.sh

#git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
%%capture
!sh setup.sh

In [0]:
%%capture
#!pip install transformers
!pip install simpletransformers

In [9]:
import sys 
sys.path.append('../preprocessing')
from preprocessing_data import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
import os
import torch
from transformers import *
nltk.download("stopwords")

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
X_train, y_train = read_normalised("../data/normalised_hindi_train.json")
X_train = np.array(X_train)
class_names = list(set(y_train))
inverse_dict = {"negative":0, "neutral":1, "positive":2}
y_inverted = [inverse_dict[i] for i in y_train]
y_inverted = np.array(y_inverted)
y_train = y_inverted

X_val, y_val = read_normalised("../data/normalised_hindi_dev.json")
X_val = np.array(X_val)
class_names = list(set(y_val))
inverse_dict = {"negative":0, "neutral":1, "positive":2}
y_inverted = [inverse_dict[i] for i in y_val]
y_inverted = np.array(y_inverted)
y_val = y_inverted
X_train[0:5]

array(['nen á visit bolest vztek smutek zmatek osam ě lost beznad ě j a nakonec jen klid Asi takhle vypad á m ů j life ...',
       '@ nehantics Haan yaar neha 😔😔 kab karega woh post 😭 Usne na sach mein photoshoot karna chahiye phir woh post karega … https // to / 5rslsbzntt',
       '@ rahulgandhi television media congress ke liye nhi h . Ye toh aapko pata chal hi gya hoga . Achha hoga ki Congress ke … https //  . co / HmH8M7PTaK',
       '@ AmitShah @ narendramodi All India me nrc lagu kare with Kashmir se dhara 370ko khatam kare ham Indian ko apse yahi umid hai',
       '@ nehr _ who @ TypoMantri @ anjanaomkashyap Pagal hai kya ? they are ’ t real issues mandir is important hindu khatre mei jo hai !'],
      dtype='<U236')

In [0]:
X_en_train = []
with open("../data/english_hinglish_train.txt", "r") as f:
    for line in f:
        line = line.strip()
        X_en_train.append(line)
X_en_train = np.array(X_en_train)

X_en_val = []
with open("../data/english_hinglish_dev.txt", "r") as f:
    for line in f:
        line = line.strip()
        X_en_val.append(line)
X_en_val = np.array(X_en_val)

In [0]:
X_es_train = []
with open("../data/hindi_hinglish_train.txt", "r") as f:
    for line in f:
        line = line.strip()
        X_es_train.append(line)
X_es_train = np.array(X_es_train)

X_es_val = []
with open("../data/hindi_hinglish_dev.txt", "r") as f:
    for line in f:
        line = line.strip()
        X_es_val.append(line)
X_es_val = np.array(X_es_val)

In [13]:
df_train = pd.DataFrame({"text": X_train, "labels": y_train})
df_val = pd.DataFrame({"text": X_val, "labels": y_val})
print(df_train)

                                                    text  labels
0      nen á visit bolest vztek smutek zmatek osam ě ...       1
1      @ nehantics Haan yaar neha 😔😔 kab karega woh p...       1
2      @ rahulgandhi television media congress ke liy...       0
3      @ AmitShah @ narendramodi All India me nrc lag...       2
4      @ nehr _ who @ TypoMantri @ anjanaomkashyap Pa...       1
...                                                  ...     ...
13995  @ anandk2012 @ railminindia @ irctcofficial @ ...       0
13996  so i couldn't get no one to cover me for my bi...       2
13997  retweet @ theskindoctor13 @ najimkhan07 aap lo...       2
13998  @ priyaverma __ @ ashutosh83b jay Jay Shree Ra...       2
13999  @ kanpuriya _ Mujhe to bhayankar ho gaya hai 🙂...       1

[14000 rows x 2 columns]


In [14]:
df_train_english = pd.DataFrame({"text": X_en_train, "labels": y_train})
df_val_english = pd.DataFrame({"text": X_en_val, "labels": y_val})
print(df_train_english)

                                                    text  labels
0      nen visit the confusion lost frustrating pain ...       1
1      @ Nehantics Haan yaar neha 😔😔 kab karega woh p...       1
2      @ Rahulgandhi television media congress ke liy...       0
3      @ Amitkshh @ narendramodi All India me nrc lag...       2
4      @ Nehr _ Hu @ Typonantri @ anjanaomkashyap Pag...       1
...                                                  ...     ...
13995  @ Anandk20l2 @ Ralminindia @ Irchtchofarfaicia...       0
13996  So I could not get anyone to cover me for my b...       2
13997  Retwet @ Theskindochtorl3 @ najimkhan07 aap lo...       2
13998  Priyverm @ __ @ ashutosh83b jay Jay Shree Ram ...       2
13999  @ Knpuriy _ Mujhe to bhayankar ho gaya hai 🙂 s...       1

[14000 rows x 2 columns]


In [15]:
df_train_spanish = pd.DataFrame({"text": X_es_train, "labels": y_train})
df_val_spanish = pd.DataFrame({"text": X_es_val, "labels": y_val})
print(df_train_spanish)

                                                    text  labels
0      nen और यात्रा भ्रम Osam का दर्द क्रोध उदासी j ...       1
1      @ nehantics Haan yaar neha 😔😔 kab karega woh p...       1
2      @ rahulgandhi television media congress ke liy...       0
3      @ AmitShah @ narendramodi All India me nrc lag...       2
4      @ nehr _ who @ TypoMantri @ anjanaomkashyap Pa...       1
...                                                  ...     ...
13995  @ anandk2012 @ railminindia @ irctcofficial @ ...       0
13996  इसलिए मैं अपने जन्मदिन के लिए मुझे कवर करने के...       2
13997  retweet @ theskindoctor13 @ najimkhan07 aap lo...       2
13998  @ priyaverma __ @ ashutosh83b jay Jay Shree Ra...       2
13999  @ kanpuriya _ Mujhe to bhayankar ho gaya hai 🙂...       1

[14000 rows x 2 columns]


In [16]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('xlmroberta', 'xlm-roberta-base', num_labels=3, args={"output_dir": "model_outputs/outputs_64_hindi_hinglish/",
    "model_cache/cache_dir": "cache_dir_64_hindi_hinglish",'learning_rate':1e-5, "train_batch_size": 64, "eval_batch_size": 64, 'num_train_epochs': 3, 'reprocess_input_data': True, 'overwrite_output_dir': True, 'evaluate_during_training': False})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:157: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:167: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


In [17]:
# Train the model
import apex
model.train_model(df_train_spanish)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.098302

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:91: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.919969Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.891313Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.990303

Running loss: 0.734211

Running loss: 0.717299
Training of xlmroberta model complete. Saved to model_outputs/outputs_64_hindi_hinglish/.


In [18]:
result, model_outputs, wrong_predictions = model.eval_model(df_val_spanish)

Features loaded from cache at cache_dir/cached_dev_xlmroberta_128_3_3000



{'mcc': 0.42245667482404303, 'eval_loss': 0.8510330890087371}


In [0]:
y_pred = np.array([np.argmax(probability) for probability in model_outputs])

In [0]:
with open("results/results_64_batch_hindi_hinglish.txt", "w") as f:
    for line in y_pred:
        f.write(str(line) + "\n")

In [21]:
from sklearn.metrics import classification_report
target_names = ['negative', 'neutral', 'positive']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.60      0.66      0.63       890
     neutral       0.56      0.50      0.53      1128
    positive       0.69      0.70      0.70       982

    accuracy                           0.61      3000
   macro avg       0.62      0.62      0.62      3000
weighted avg       0.61      0.61      0.61      3000



In [0]:
#predictions, raw_outputs = model.predict(['Some arbitary sentence']